In [4]:
import pandas as pd
import numpy as np
import os
import tqdm as tqdm
# correlation between two variables
from scipy.stats import pearsonr
import gc
gc.collect()

0

In [5]:
def read_data(path):
    """reads all the data in current directory and add a column for the file name for each one. using pd.concat, concats all the data into one dataframe"""
    df = pd.DataFrame()
    for file in os.listdir(path):
        if file.endswith(".csv"):
            filez = os.path.join(path, file)
            with open(filez) as fp:
                width = len(fp.readline().strip().split(','))
                dtypes = {i: np.float16 for i in range(width)} # ########## changed to bool 8 
                # update the last column's dtype
                dtypes[0] = np.str0
                dtypes[1] = np.bool8
                # reset the read position of the file pointer
                fp.seek(400000)
                temp = pd.read_csv(fp, dtype=dtypes, low_memory=False)
            print(file, ":", temp.shape)
            df = pd.concat([df, temp], axis=1)
            print(df.shape)
            df.drop_duplicates(inplace=True, )
            print(df.shape)
            # df.to_csv(f"{path}{file}", index=False,)

    return df

In [6]:
def correlation(df):
    """calculates the correlation between the columns and returns a dataframe with the correlation"""
    corr = df.corr()
    return corr


In [7]:
df = read_data(".")

NCP.csv : (404965, 288)


UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
# corr = correlation(df)
# corr

In [48]:
# visualize the correlation
import seaborn as sns
import matplotlib.pyplot as plt

# only show high correlation
corr = corr[abs(corr) > 0.8]


# disregard the diagonal
corr = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
# filter out NaN columns
corr = corr.dropna(axis=1, how="all")
corr = corr.dropna(axis=0, how="all")

corr

/tmp/ipykernel_40389/1686278919.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  corr = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))


,ENAC.csv_label,Mismatch.csv_label,DACC.csv_label,DPCP.csv_label,DPCP.csv_AA_Duplex stability:(freeenergy),DPCP.csv_CC_Duplex stability:(freeenergy),DPCP.csv_CG_Duplex stability:(freeenergy),DPCP.csv_GG_Duplex stability:(freeenergy),DPCP.csv_TT_Duplex stability:(freeenergy),DPCP.csv_AA_Duplex tability(disruptenergy),...,TPCP.csv_TCT_Nucleosome-Rigid,TPCP.csv_TGA_Nucleosome-Rigid,TPCP.csv_TGC_Nucleosome-Rigid,TPCP.csv_TGG_Nucleosome-Rigid,TPCP.csv_TGT_Nucleosome-Rigid,TPCP.csv_TTA_Nucleosome-Rigid,TPCP.csv_TTC_Nucleosome-Rigid,TPCP.csv_TTG_Nucleosome-Rigid,TPCP.csv_TTT_Nucleosome-Rigid,PSTNPds.csv_label
PSTNPss.csv_label,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
NCP.csv_label,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
NCP.csv_NCP.F1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCP.csv_NCP.F2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NCP.csv_NCP.F4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TPCP.csv_TGT_MW-kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN
TPCP.csv_TTA_MW-kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
TPCP.csv_TTC_MW-kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN
TPCP.csv_TTG_MW-kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


In [49]:
# drop the columns that are correlated with each other
col_to_drop = corr.columns.to_list()
col_to_drop += corr.index.to_list()

print("Number of columns to drop:", len(set(col_to_drop)))

col_to_drop = set(col_to_drop)

Number of columns to drop: 5646


In [7]:

df.drop(col_to_drop, axis=1, inplace=True)
print("Size of df in kb:", df.memory_usage().sum() / 1024)

Size of df in kb: 12406.375


In [52]:
def shaper(col):
    c = 0   
    col_to_drop = corr.columns.to_list()
    col_to_drop += corr.index.to_list()
    for i in set(col_to_drop):
        if i.startswith(col):
            c += 1
    return c


# print the high correlation columns and their correlation value
j = 0
dic = {}
for col in corr.columns.to_list():
    for index, value in corr[col].iteritems():
        if abs(value) > 0.8:
            if index.endswith("label") and col.endswith("label"):
                continue
            elif col.split("_")[0] == index.split("_")[0]:
                continue
            else:
                # print(f"{col} - {index} : {value}")
                if (col.split("_")[0]+"_"+index.split("_")[0] not in dic):
                    dic[col.split("_")[0]+"_"+index.split("_")[0]] = 1
                elif (index.split("_")[0]+"_"+col.split("_")[0] in dic):
                    # dic[col.split("_")[0]+"_"+index.split("_")[0]] += 1
                    print("###############")
                    continue
                else:
                    dic[col.split("_")[0]+"_"+index.split("_")[0]] += 1
            j += 1
for i in dic:
    dic[i] = dic[i] / (shaper(i.split("_")[0]) + shaper(i.split("_")[1]))
print(f"total: {j}")            
print(dic)

/tmp/ipykernel_40389/776041458.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in corr[col].iteritems():


total: 22400
{'DPCP.csv_Mismatch.csv': 0.4630030289917785, 'Geary.csv_DACC.csv': 0.11087420042643924, 'TACC.csv_Geary.csv': 0.026373626373626374, 'PCPseDNC.csv_Mismatch.csv': 0.25806451612903225, 'PCPseDNC.csv_DPCP.csv': 0.99137187230371, 'CKSNAP.csv_Mismatch.csv': 0.4864864864864865, 'CKSNAP.csv_DPCP.csv': 0.9888123924268503, 'PCPseTNC.csv_DPCP.csv': 0.12351945854483926, 'SCPseTNC.csv_DPCP.csv': 0.12341504649196956, 'SCPseTNC.csv_TACC.csv': 0.02564102564102564, 'ASDC.csv_Mismatch.csv': 1.2105263157894737, 'ASDC.csv_DPCP.csv': 0.2541196877710321, 'RCKmer.csv_TACC.csv': 0.17647058823529413, 'RCKmer.csv_DACC.csv': 0.03225806451612903, 'Subsequence.csv_DPCP.csv': 0.12351945854483926, 'DBE.csv_NCP.csv': 0.6622516556291391, 'NMBroto.csv_DACC.csv': 0.3135593220338983, 'NMBroto.csv_Geary.csv': 0.40610687022900765, 'NMBroto.csv_DPCP.csv': 0.17410358565737052, 'NMBroto.csv_PCPseDNC.csv': 0.013043478260869565, 'NMBroto.csv_CKSNAP.csv': 0.012711864406779662, 'NMBroto.csv_PCPseTNC.csv': 0.00724637

In [51]:
# sort dic on value
dic = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1], reverse=True)}
dic

{'Moran.csv_Geary.csv': 1.5292792792792793,
 'ASDC.csv_Mismatch.csv': 1.2105263157894737,
 'TPCP.csv_DPCP.csv': 1.1421121251629727,
 'PCPseDNC.csv_DPCP.csv': 0.99137187230371,
 'PseDNC.csv_DPCP.csv': 0.99137187230371,
 'CKSNAP.csv_DPCP.csv': 0.9888123924268503,
 'TPCP.csv_PCPseTNC.csv': 0.920863309352518,
 'TPCP.csv_Subsequence.csv': 0.920863309352518,
 'TPCP.csv_PseKNC.csv': 0.920863309352518,
 'TPCP.csv_PseEIIP.csv': 0.920863309352518,
 'TPCP.csv_Kmer.csv': 0.920863309352518,
 'TPCP.csv_SCPseTNC.csv': 0.9186602870813397,
 'DBE.csv_NCP.csv': 0.6622516556291391,
 'EIIP.csv_binary.csv': 0.4950980392156863,
 'PseKNC.csv_PCPseTNC.csv': 0.49230769230769234,
 'PseKNC.csv_Subsequence.csv': 0.49230769230769234,
 'PseEIIP.csv_PCPseTNC.csv': 0.49230769230769234,
 'PseEIIP.csv_Subsequence.csv': 0.49230769230769234,
 'Kmer.csv_PCPseTNC.csv': 0.49230769230769234,
 'Kmer.csv_Subsequence.csv': 0.49230769230769234,
 'CKSNAP.csv_Mismatch.csv': 0.4864864864864865,
 'PseKNC.csv_SCPseTNC.csv': 0.48484848

In [34]:
# save df to file
df.to_csv("df.csv", index=False)

In [12]:
df.shape

(500, 3176)

In [24]:
c = 0   
col_to_drop = corr.columns.to_list()
col_to_drop += corr.index.to_list()
for i in set(col_to_drop):
    if i.startswith("DPCP"):
        c += 1
print(c)

2299


In [47]:
a = {'Moran.csv_Geary.csv': 1.5292792792792793, 
#  'TPCP.csv_DPCP.csv': 1.1421121251629727, # delete TPCP and DPCP
#  'PCPseDNC.csv_DPCP.csv': 0.99137187230371,
#  'PseDNC.csv_DPCP.csv': 0.99137187230371,
#  'CKSNAP.csv_DPCP.csv': 0.9862660944206009,
#  'ASDC.csv_Mismatch.csv': 0.9583333333333334, # DELETE ASDC
#  'TPCP.csv_PCPseTNC.csv': 0.920863309352518,
#  'TPCP.csv_Subsequence.csv': 0.920863309352518,
#  'TPCP.csv_PseEIIP.csv': 0.920863309352518,
#  'TPCP.csv_Kmer.csv': 0.920863309352518,
#  'TPCP.csv_PseKNC.csv': 0.9197604790419162,
 'TPCP.csv_SCPseTNC.csv': 0.8951048951048951,}
s = {}


'Moran.csv_Geary.csv': 1.5292792792792793,
 'ASDC.csv_Mismatch.csv': 1.2105263157894737,
 'TPCP.csv_DPCP.csv': 1.1421121251629727,
 'PCPseDNC.csv_DPCP.csv': 0.99137187230371,
 'PseDNC.csv_DPCP.csv': 0.99137187230371,
 'CKSNAP.csv_DPCP.csv': 0.9888123924268503,
 'TPCP.csv_PCPseTNC.csv': 0.920863309352518,
 'TPCP.csv_Subsequence.csv': 0.920863309352518,
 'TPCP.csv_PseKNC.csv': 0.920863309352518,
 'TPCP.csv_PseEIIP.csv': 0.920863309352518,
 'TPCP.csv_Kmer.csv': 0.920863309352518,
 'TPCP.csv_SCPseTNC.csv': 0.9186602870813397,
for i in a:
    f = i.split("_")[0]
    sec = i.split("_")[1]
    if f in s:
        s[f] += 1
    elif sec in s:
        s[sec] += 1
    elif f not in s:
        s[f] = 1
    else:
        s[sec] = 1

# sort s on value
s = {k: v for k, v in sorted(s.items(), key=lambda item: item[1], reverse=True)}
s



{'TPCP.csv': 6,
 'Moran.csv': 1,
 'PCPseDNC.csv': 1,
 'PseDNC.csv': 1,
 'CKSNAP.csv': 1,
 'ASDC.csv': 1}

In [ ]:
{'Moran.csv_Geary.csv': 1.5292792792792793,
 'TPCP.csv_DPCP.csv': 1.1421121251629727,
 'PCPseDNC.csv_DPCP.csv': 0.99137187230371,
 'PseDNC.csv_DPCP.csv': 0.99137187230371,
 'CKSNAP.csv_DPCP.csv': 0.9862660944206009,
 'ASDC.csv_Mismatch.csv': 0.9583333333333334,
 'TPCP.csv_PCPseTNC.csv': 0.920863309352518,
 'TPCP.csv_Subsequence.csv': 0.920863309352518,
 'TPCP.csv_PseEIIP.csv': 0.920863309352518,
 'TPCP.csv_Kmer.csv': 0.920863309352518,
 'TPCP.csv_PseKNC.csv': 0.9197604790419162,
 'TPCP.csv_SCPseTNC.csv': 0.8951048951048951,
 'DBE.csv_NCP.csv': 0.6622516556291391,
 'DPCP.csv_Mismatch.csv': 0.5034542314335061,
 'EIIP.csv_binary.csv': 0.4950980392156863,
 'Subsequence.csv_PCPseTNC.csv': 0.49230769230769234,
 'PseEIIP.csv_PCPseTNC.csv': 0.49230769230769234,
 'PseEIIP.csv_Subsequence.csv': 0.49230769230769234,
 'Kmer.csv_PCPseTNC.csv': 0.49230769230769234,
 'Kmer.csv_Subsequence.csv': 0.49230769230769234,
 'Kmer.csv_PseEIIP.csv': 0.49230769230769234,
 'PseKNC.csv_PCPseTNC.csv': 0.48854961832061067,
 'PseKNC.csv_Subsequence.csv': 0.48854961832061067,
 'PseEIIP.csv_PseKNC.csv': 0.48854961832061067,
 'Kmer.csv_PseKNC.csv': 0.48854961832061067,
 'PseDNC.csv_PCPseDNC.csv': 0.47368421052631576,
 'SCPseTNC.csv_PCPseTNC.csv': 0.4155844155844156,
 'Subsequence.csv_SCPseTNC.csv': 0.4155844155844156,
 'PseEIIP.csv_SCPseTNC.csv': 0.4155844155844156,
 'Kmer.csv_SCPseTNC.csv': 0.4155844155844156,
 'PseKNC.csv_SCPseTNC.csv': 0.4129032258064516,
 'ASDC.csv_CKSNAP.csv': 0.39473684210526316,
 'CKSNAP.csv_Mismatch.csv': 0.375,
 'Moran.csv_NMBroto.csv': 0.3701863354037267,
 'NMBroto.csv_Geary.csv': 0.33043478260869563,
 'CKSNAP.csv_PCPseDNC.csv': 0.32,
 'PseDNC.csv_CKSNAP.csv': 0.32,
 'ASDC.csv_DPCP.csv': 0.2541196877710321,
 'SCPseTNC.csv_TACC.csv': 0.22718446601941747,
 'PCPseDNC.csv_Mismatch.csv': 0.2222222222222222,
 'PseDNC.csv_Mismatch.csv': 0.2222222222222222,
 'NMBroto.csv_DACC.csv': 0.1689497716894977,
 'NMBroto.csv_DPCP.csv': 0.16428571428571428,
 'ASDC.csv_PCPseDNC.csv': 0.15384615384615385,
 'PseDNC.csv_ASDC.csv': 0.15384615384615385,
 'TPCP.csv_RCKmer.csv': 0.12355212355212356,
 'PCPseTNC.csv_DPCP.csv': 0.12351945854483926,
 'Subsequence.csv_DPCP.csv': 0.12351945854483926,
 'PseEIIP.csv_DPCP.csv': 0.12351945854483926,
 'Kmer.csv_DPCP.csv': 0.12351945854483926,
 'PseKNC.csv_DPCP.csv': 0.12346723044397463,
 'SCPseTNC.csv_DPCP.csv': 0.12227805695142378,
 'Subsequence.csv_RCKmer.csv': 0.1095890410958904,
 'PseEIIP.csv_RCKmer.csv': 0.1095890410958904,
 'Kmer.csv_RCKmer.csv': 0.1095890410958904,
 'Moran.csv_DACC.csv': 0.10556621880998081,
 'Geary.csv_DACC.csv': 0.09980806142034548,
 'RCKmer.csv_PCPseTNC.csv': 0.0821917808219178,
 'PseKNC.csv_RCKmer.csv': 0.08108108108108109,
 'RCKmer.csv_CKSNAP.csv': 0.07692307692307693,
 'RCKmer.csv_SCPseTNC.csv': 0.061855670103092786,
 'Moran.csv_TACC.csv': 0.040229885057471264,
 'TACC.csv_Geary.csv': 0.035632183908045977,
 'TPCP.csv_PCPseDNC.csv': 0.030456852791878174,
 'TPCP.csv_PseDNC.csv': 0.030456852791878174,
 'TPCP.csv_CKSNAP.csv': 0.03,
 'PCPseTNC.csv_PCPseDNC.csv': 0.023809523809523808,
 'Subsequence.csv_PCPseDNC.csv': 0.023809523809523808,
 'PseEIIP.csv_PCPseDNC.csv': 0.023809523809523808,
 'Kmer.csv_PCPseDNC.csv': 0.023809523809523808,
 'PseDNC.csv_PCPseTNC.csv': 0.023809523809523808,
 'PseDNC.csv_Subsequence.csv': 0.023809523809523808,
 'PseDNC.csv_PseEIIP.csv': 0.023809523809523808,
 'PseDNC.csv_Kmer.csv': 0.023809523809523808,
 'PseKNC.csv_PCPseDNC.csv': 0.023529411764705882,
 'PseDNC.csv_PseKNC.csv': 0.023529411764705882,
 'TPCP.csv_NMBroto.csv': 0.021238938053097345,
 'PCPseTNC.csv_CKSNAP.csv': 0.020833333333333332,
 'Subsequence.csv_CKSNAP.csv': 0.020833333333333332,
 'PseEIIP.csv_CKSNAP.csv': 0.020833333333333332,
 'Kmer.csv_CKSNAP.csv': 0.020833333333333332,
 'RCKmer.csv_TACC.csv': 0.020737327188940093,
 'PseKNC.csv_CKSNAP.csv': 0.020618556701030927,
 'NMBroto.csv_RCKmer.csv': 0.018970189701897018,
 'SCPseTNC.csv_PCPseDNC.csv': 0.018518518518518517,
 'PseDNC.csv_SCPseTNC.csv': 0.018518518518518517,
 'SCPseTNC.csv_CKSNAP.csv': 0.016666666666666666,
 'RCKmer.csv_DACC.csv': 0.011764705882352941,
 'NMBroto.csv_SCPseTNC.csv': 0.008888888888888889,
 'NMBroto.csv_PCPseDNC.csv': 0.007894736842105263,
 'PseDNC.csv_NMBroto.csv': 0.007894736842105263,
 'NMBroto.csv_CKSNAP.csv': 0.007653061224489796,
 'PseKNC.csv_NMBroto.csv': 0.00702576112412178,
 'NMBroto.csv_PCPseTNC.csv': 0.004694835680751174,
 'NMBroto.csv_Subsequence.csv': 0.004694835680751174,
 'PseEIIP.csv_NMBroto.csv': 0.004694835680751174,
 'Kmer.csv_NMBroto.csv': 0.004694835680751174,
 'NMBroto.csv_MMI.csv': 0.0027548209366391185}